In [16]:
import os
import pandas as pd
from datasets import Dataset, DatasetDict
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model
import json
import re

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# Step 1: Load Dataset
dataset_path = '../.dataset/hdfs/train.csv'

In [17]:
data = pd.read_csv(dataset_path)#, nrows=10000)
data_normal = data[data["Label"] == 0]  # Filter out anomalies
data_anomaly = data[data["Label"] == 1] 

In [19]:
regex = [
        r"(?<=blk_)[-\d]+", # block_id
        r'\d+\.\d+\.\d+\.\d+',  # IP
        r"(/[-\w]+)+",  # file path
        #r'(?<=[^A-Za-z0-9])(\-?\+?\d+)(?=[^A-Za-z0-9])|[0-9]+$',  # Numbers
        ]

def apply_regex(line):
    for r in regex:
        line = re.sub(r, '<*>', line)
    return line

In [20]:
data_normal['Content'] = data_normal['Content'].apply(apply_regex)

/tmp/ipykernel_560610/148702825.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_normal['Content'] = data_normal['Content'].apply(apply_regex)
